In [1]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm

In [2]:
np.random.seed(42)

# Dimensions Set-up

In [3]:
periods = pd.date_range("2025-01-01", periods=12, freq="MS").strftime("%Y-%m")

### Plant - Cost Centers Configuration

In [4]:
n_plant = 6
base_cc_count = 10

# PLANTS SIZE
raw_scale = np.random.lognormal(mean=0.0, sigma=0.25, size=n_plant)
scale = raw_scale / raw_scale.mean()   # normalize around 1.0

# PLANTS BIASES
# automation: random factor representing plant automation level (-1 to 1)
# dm_bias: positive bias for high-automation plants (more efficient materials usage)
# dl_bias: negative bias for high-automation plants (less labor needed)
automation = np.random.uniform(-1, 1, n_plant)
dm_bias = np.clip(0.03 * automation, -0.05, 0.05)
dl_bias = np.clip(-0.03 * automation, -0.05, 0.05)

# PRODUCTION vs SUPPORTING CC RATIO
# Higher automation = more supporting CCs needed (less production)
# Larger plants = more supporting CCs needed (less production)
prod_cc_ratio = np.clip(0.75 - 0.15 * automation - 0.10 * (scale - 1), 0.60, 0.85)

plants = {
    f"PLANT {i+1:d}": {
        "SCALE": round(scale[i], 3),
        "DM_BIAS": round(dm_bias[i], 3),
        "DL_BIAS": round(dl_bias[i], 3),
        "CC_COUNT": max(3, int(np.random.poisson(base_cc_count*round(scale[i], 3)))),
        "PROD_CC_RATIO": round(prod_cc_ratio[i], 3)
    }
    for i in range(n_plant)
}
plants

{'PLANT 1': {'SCALE': np.float64(1.026),
  'DM_BIAS': np.float64(-0.027),
  'DL_BIAS': np.float64(0.027),
  'CC_COUNT': 14,
  'PROD_CC_RATIO': np.float64(0.85)},
 'PLANT 2': {'SCALE': np.float64(0.875),
  'DM_BIAS': np.float64(0.022),
  'DL_BIAS': np.float64(-0.022),
  'CC_COUNT': 7,
  'PROD_CC_RATIO': np.float64(0.653)},
 'PLANT 3': {'SCALE': np.float64(1.065),
  'DM_BIAS': np.float64(0.006),
  'DL_BIAS': np.float64(-0.006),
  'CC_COUNT': 8,
  'PROD_CC_RATIO': np.float64(0.713)},
 'PLANT 4': {'SCALE': np.float64(1.326),
  'DM_BIAS': np.float64(0.012),
  'DL_BIAS': np.float64(-0.012),
  'CC_COUNT': 13,
  'PROD_CC_RATIO': np.float64(0.655)},
 'PLANT 5': {'SCALE': np.float64(0.854),
  'DM_BIAS': np.float64(-0.029),
  'DL_BIAS': np.float64(0.029),
  'CC_COUNT': 6,
  'PROD_CC_RATIO': np.float64(0.85)},
 'PLANT 6': {'SCALE': np.float64(0.854),
  'DM_BIAS': np.float64(0.028),
  'DL_BIAS': np.float64(-0.028),
  'CC_COUNT': 9,
  'PROD_CC_RATIO': np.float64(0.624)}}

In [5]:
cc_map = {}
for i, plant in enumerate(plants.keys()):
    n_cc = plants[plant]['CC_COUNT']
    n_prod = max(1, int(n_cc * plants[plant]['PROD_CC_RATIO']))
    n_support = n_cc - n_prod
    
    prod_ccs = [f"CC{i+1:03d}P{j:02d}" for j in range(1, n_prod + 1)]
    support_ccs = [f"CC{i+1:03d}S{j:02d}" for j in range(1, n_support + 1)]
    cc_map[plant] = prod_ccs + support_ccs

cc_df = (
    pd.DataFrame(
        [
            (plant, cc, "Production" if "P" in cc else "Supporting")
            for plant, ccs in cc_map.items()
            for cc in ccs
        ],
        columns=["PLANT", "COST_CENTER", "CAT_CC"]
    )
)
cc_df

,PLANT,COST_CENTER,CAT_CC
0,PLANT 1,CC001P01,Production
1,PLANT 1,CC001P02,Production
2,PLANT 1,CC001P03,Production
3,PLANT 1,CC001P04,Production
4,PLANT 1,CC001P05,Production
5,PLANT 1,CC001P06,Production
6,PLANT 1,CC001P07,Production
7,PLANT 1,CC001P08,Production
8,PLANT 1,CC001P09,Production
9,PLANT 1,CC001P10,Production


### COA Configuration

In [6]:
cost_elements = pd.DataFrame(
    [
        ("DM", "RM"),
        ("DM", "PM"),
        ("DL", "DL"),
        ("OH", "OH_FIX"),
        ("OH", "OH_VAR"),
    ],
    columns=["CAT_GL", "COST_ELEMENT"]
)
cost_elements

,CAT_GL,COST_ELEMENT
0,DM,RM
1,DM,PM
2,DL,DL
3,OH,OH_FIX
4,OH,OH_VAR


In [7]:
elasticity_range = {
    "RM": (0.85, 0.95),
    "PM": (0.85, 0.95),
    "DL": (0.60, 0.80),
    "OH_VAR": (0.90, 1.05),
    "OH_FIX": (0.00, 0.00),
}

elasticity_table = (
    pd.MultiIndex.from_product(
        [plants.keys(), elasticity_range.keys()],
        names=["PLANT", "COST_ELEMENT"]
    )
    .to_frame(index=False)
)

elasticity_table["ELASTICITY"] = elasticity_table["COST_ELEMENT"].apply(
    lambda x: np.random.uniform(*elasticity_range[x])
)
elasticity_table

,PLANT,COST_ELEMENT,ELASTICITY
0,PLANT 1,RM,0.940932
1,PLANT 1,PM,0.875878
2,PLANT 1,DL,0.732504
3,PLANT 1,OH_VAR,0.946757
4,PLANT 1,OH_FIX,0.000000
5,PLANT 2,RM,0.904671
6,PLANT 2,PM,0.868485
7,PLANT 2,DL,0.793917
8,PLANT 2,OH_VAR,1.016270
9,PLANT 2,OH_FIX,0.000000


# Fact Table Frame

In [8]:
df = cc_df.assign(key=1).merge(
    pd.DataFrame({"PERIOD": periods, "key": 1}),
    on="key"
).drop("key", axis=1)
df

,PLANT,COST_CENTER,CAT_CC,PERIOD
0,PLANT 1,CC001P01,Production,2025-01
1,PLANT 1,CC001P01,Production,2025-02
2,PLANT 1,CC001P01,Production,2025-03
3,PLANT 1,CC001P01,Production,2025-04
4,PLANT 1,CC001P01,Production,2025-05
...,...,...,...,...
679,PLANT 6,CC006S04,Supporting,2025-08
680,PLANT 6,CC006S04,Supporting,2025-09
681,PLANT 6,CC006S04,Supporting,2025-10
682,PLANT 6,CC006S04,Supporting,2025-11


In [9]:
plant_df = pd.DataFrame.from_dict(plants, orient="index")
plant_df.index.name = "PLANT"

# df = df.merge(plant_df, on="Plant")

# df = df.merge(cost_elements, how="cross")
# df
plant_df

,SCALE,DM_BIAS,DL_BIAS,CC_COUNT,PROD_CC_RATIO
PLANT,,,,,
PLANT 1,1.026,-0.027,0.027,14,0.850
PLANT 2,0.875,0.022,-0.022,7,0.653
PLANT 3,1.065,0.006,-0.006,8,0.713
PLANT 4,1.326,0.012,-0.012,13,0.655
PLANT 5,0.854,-0.029,0.029,6,0.850
PLANT 6,0.854,0.028,-0.028,9,0.624


# Activity & Qty Baseline

In [10]:
pp = df[["PERIOD", "PLANT"]].drop_duplicates()

pp["ACTIVITY_INDEX"] = np.clip(
    np.random.normal(1.0, 0.1, len(pp)),
    0.8, 1.2
)

pp = pp.merge(plant_df, on="PLANT")
pp

,PERIOD,PLANT,ACTIVITY_INDEX,SCALE,DM_BIAS,DL_BIAS,CC_COUNT,PROD_CC_RATIO
0,2025-01,PLANT 1,1.058212,1.026,-0.027,0.027,14,0.850
1,2025-02,PLANT 1,1.088775,1.026,-0.027,0.027,14,0.850
2,2025-03,PLANT 1,1.089433,1.026,-0.027,0.027,14,0.850
3,2025-04,PLANT 1,1.075500,1.026,-0.027,0.027,14,0.850
4,2025-05,PLANT 1,0.979283,1.026,-0.027,0.027,14,0.850
...,...,...,...,...,...,...,...,...
67,2025-08,PLANT 6,0.957850,0.854,0.028,-0.028,9,0.624
68,2025-09,PLANT 6,1.033982,0.854,0.028,-0.028,9,0.624
69,2025-10,PLANT 6,0.999262,0.854,0.028,-0.028,9,0.624
70,2025-11,PLANT 6,1.076730,0.854,0.028,-0.028,9,0.624


In [11]:
base_qty = 10_000
cv = 0.12  # Coefficient of variation (adjust based on your volatility assumptions)
bound_pct = 0.30  # ±30% bounds

mean_qty = base_qty * pp["SCALE"] * pp["ACTIVITY_INDEX"]
pp["QTY"] = np.clip(
    np.random.normal(mean_qty, mean_qty * cv),
    mean_qty * (1 - bound_pct),
    mean_qty * (1 + bound_pct)
).astype(int)
# df = df.merge(pp, on=["PERIOD", "PLANT"])
# df
pp

,PERIOD,PLANT,ACTIVITY_INDEX,SCALE,DM_BIAS,DL_BIAS,CC_COUNT,PROD_CC_RATIO,QTY
0,2025-01,PLANT 1,1.058212,1.026,-0.027,0.027,14,0.850,9847
1,2025-02,PLANT 1,1.088775,1.026,-0.027,0.027,14,0.850,12207
2,2025-03,PLANT 1,1.089433,1.026,-0.027,0.027,14,0.850,10102
3,2025-04,PLANT 1,1.075500,1.026,-0.027,0.027,14,0.850,12867
4,2025-05,PLANT 1,0.979283,1.026,-0.027,0.027,14,0.850,11741
...,...,...,...,...,...,...,...,...,...
67,2025-08,PLANT 6,0.957850,0.854,0.028,-0.028,9,0.624,8906
68,2025-09,PLANT 6,1.033982,0.854,0.028,-0.028,9,0.624,8350
69,2025-10,PLANT 6,0.999262,0.854,0.028,-0.028,9,0.624,9329
70,2025-11,PLANT 6,1.076730,0.854,0.028,-0.028,9,0.624,10348


In [ ]:
group_total = 1_000_000

# include ACTIVITY_INDEX so monthly plant totals vary with activity
mix = pp[["PERIOD", "PLANT", "SCALE", "DM_BIAS", "DL_BIAS", "ACTIVITY_INDEX"]].copy()

mix["DM_RATIO"] = np.random.uniform(0.55, 0.65, len(mix)) + mix["DM_BIAS"]
mix["DL_RATIO"] = np.random.uniform(0.20, 0.30, len(mix)) + mix["DL_BIAS"]
mix["OH_RATIO"] = 1 - mix["DM_RATIO"] - mix["DL_RATIO"]

# partial dependence: keep 30% of budget fixed, 70% responsive to activity
mix["PLANT_TOTAL"] = group_total * mix["SCALE"] * (0.7 * mix["ACTIVITY_INDEX"] + 0.3)

df = df.merge(
    mix[["PERIOD", "PLANT", "PLANT_TOTAL", "DM_RATIO", "DL_RATIO", "OH_RATIO"]],
    on=["PERIOD", "PLANT"]
)
df

In [ ]:
filter = (df['Plant'] == 'PLANT 1')
df.loc[filter,'Qty'].nunique()

In [ ]:
df["Category_Ratio"] = np.select(
    [
        df["Cost_Category"] == "DM",
        df["Cost_Category"] == "DL",
        df["Cost_Category"] == "OH",
    ],
    [
        df["DM_ratio"],
        df["DL_ratio"],
        df["OH_ratio"],
    ],
)

ce_count = cost_elements.groupby("Cost_Category").size()
df["CE_Count"] = df["Cost_Category"].map(ce_count)

cc_count = cc_df.groupby("Plant").size()
df["CC_Count"] = df["Plant"].map(cc_count)

df["Plan_Amount"] = (
    df["Plant_Total"]
    * df["Category_Ratio"]
    / df['CC_Count']
    / df['CE_Count']
    * np.random.lognormal(0, 0.15, len(df))
)
df

In [ ]:
df["Noise"] = np.clip(
    np.random.normal(0, 0.02, len(df)),
    -0.05, 0.05
)

df["Adjustment"] = np.where(
    np.random.rand(len(df)) < 0.25,
    np.random.uniform(0.01, 0.05, len(df)),
    0
)

In [ ]:
df['Plan_Amount'].hist()

In [ ]:
df = df.merge(
    elasticity_table,
    on=["Plant", "Cost_Element"],
    how="left"
)

In [ ]:
df

In [ ]:
activity_effect = np.where(
    df["Cost_Element"] == "OH_FIX",
    1.0,
    np.where(
        df["Cost_Element"] == "OH_VAR",
        df["Activity_Index"],
        df["Activity_Index"] ** df["Elasticity"]
    )
)

df["Actual_Amount"] = (
    df["Plan_Amount"]
    * activity_effect
    * (1 + df["Noise"] + df["Adjustment"])
)

# df["Variance"] = df["Actual_Amount"] - df["Plan_Amount"]

In [ ]:
df = df[[
    "Period", "Plant", "Cost_Center",
    "Cost_Category", "Cost_Element", "Plant_Total",
    "Plan_Amount", "Actual_Amount", "Qty"
]].round(2)

In [ ]:
df

In [ ]:
filter_plant = (df['Plant'] == 'PLANT_A')
filter_period = (df['Period'] == '2024-01')
df.loc[filter_plant,['Period', 'Plant_Total', 'Plan_Amount', 'Actual_Amount']].groupby('Period').sum()